The reason: 

In [1]:
# python code

Can I include pics?

Results for 2020 Election:

![Alt text](images/Screenshot%202024-04-11%20at%203.54.02%20PM.png)

Results for 2016 Election:

![Alt text](images/Screenshot%202024-04-11%20at%203.54.18%20PM.png)

Results for 2012 Election:

![Alt text](images/Screenshot%202024-04-11%20at%203.54.22%20PM.png)